In [17]:
import os
%pwd
os.chdir("../")
%pwd

'C:\\Users\\zainulabideen\\Desktop'

In [18]:
directory_path = r"C:\Users\zainulabideen\Desktop\Chest_Cancer_Classification_Mlflow_and_DVC"
os.chdir(directory_path)
%pwd

'C:\\Users\\zainulabideen\\Desktop\\Chest_Cancer_Classification_Mlflow_and_DVC'

In [72]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
 #   train_training_data: Path
 #   valid_training_data: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
 #   test_training_data: Path
    params_image_size: list

In [73]:

from Cnn_Chest_Cancer_Classification.constants import *
from Cnn_Chest_Cancer_Classification.utils.common import read_yaml, create_directories
import tensorflow as tf

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-Scan-data")
        create_directories([
            Path(training.root_dir)
        ])
     #   valid_training_data = os.path.join(self.config.data_ingestion.data, "valid")
     #   train_training_data = os.path.join(self.config.data_ingestion.data, "train")
     #   test_training_data = os.path.join(self.config.data_ingestion.data, "test")

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
      #      valid_training_data=Path(valid_training_data),
      #      train_training_data=Path(train_training_data),
            training_data=Path(training_data),
      #      test_training_data=Path(test_training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [80]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1. / 255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            #directory=valid_data_dir,
            shuffle=False,
            **dataflow_kwargs
        )
        

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
           # directory=train_data_dir,
            shuffle=True,
            **dataflow_kwargs
        )

        for images, labels in self.train_generator:
            print("Batch shape - Images:", images.shape, "Labels:", labels.shape)
            break  # Break after printing the first batch for inspection

# Example for validation generator
        for images, labels in self.valid_generator:
            print("Batch shape - Images:", images.shape, "Labels:", labels.shape)
            break 
        
  

    

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [81]:

try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-03-18 20:39:03,751: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-18 20:39:03,755: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-18 20:39:03,756: INFO: common: created directory at: artifacts]
[2024-03-18 20:39:03,758: INFO: common: created directory at: artifacts\training]
[2024-03-18 20:39:04,093: WARNING: hdf5_format: No training configuration found in the save file, so the model was *not* compiled. Compile it manually.]
Found 343 images belonging to 2 classes.
Found 343 images belonging to 2 classes.
Batch shape - Images: (16, 224, 224, 3) Labels: (16, 2)
Batch shape - Images: (16, 224, 224, 3) Labels: (16, 2)


ValueError: in user code:

    File "c:\Users\zainulabideen\AppData\Local\anaconda3\envs\cancer_project\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\zainulabideen\AppData\Local\anaconda3\envs\cancer_project\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\zainulabideen\AppData\Local\anaconda3\envs\cancer_project\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\zainulabideen\AppData\Local\anaconda3\envs\cancer_project\lib\site-packages\keras\engine\training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\zainulabideen\AppData\Local\anaconda3\envs\cancer_project\lib\site-packages\keras\engine\training.py", line 1109, in compute_loss


In [82]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [83]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-Scan-data")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [84]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [85]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [86]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-03-18 20:43:23,769: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-18 20:43:23,772: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-18 20:43:23,773: INFO: common: created directory at: artifacts]
[2024-03-18 20:43:23,776: INFO: common: created directory at: artifacts\training]
Found 68 images belonging to 2 classes.
Found 275 images belonging to 2 classes.
17/17 [==============================] - 35s 2s/step - loss: 17.3406 - accuracy: 0.5058 - val_loss: 0.7795 - val_accuracy: 0.6406
